<a href="https://colab.research.google.com/github/peterpanw/Imageclassifier/blob/main/Resnet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import datasets, layers, Sequential, optimizers, metrics   # Sequential 序贯模型
import os    # os库  用于访问操作系统的标准库 处理文件和目录
import numpy as np

In [2]:
tf.random.set_seed(0)
np.random.seed(0)
os.environ['TF_CPP_MIN_LEVEL'] = '2'
assert tf.__version__.startswith('2.')  # assert<表达式>  用于测试<表达式>的值，如果值为true正常通过，值为false则报错"AssertError"

## 配置超参数
batch_size = 128
optimizer = optimizers.Adam(lr=0.0001)     #优化器
epochs = 20

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [3]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
print("train_shape:", x_train.shape, y_train.shape)
y_train = tf.squeeze(y_train,  axis=1)
y_test = tf.squeeze(y_test, axis=1)
print("train_shape:", x_train.shape, y_train.shape)

170508288/170498071 [==============================] - 6s 0us/step
train_shape: (50000, 32, 32, 3) (50000, 1)
train_shape: (50000, 32, 32, 3) (50000,)


In [4]:
def preprocess(x,y):
    x = tf.cast(x, dtype=tf.float32)/255.
    y = tf.cast(y, dtype=tf.int32)
    return x, y
train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_db = train_db.map(preprocess).shuffle(50000).batch(batch_size)
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_db = test_db.map(preprocess).batch(batch_size)

In [5]:
####   构建 简单层
class BasicBlock(layers.Layer):
    def __init__(self, filter_num, strides=1):
        super(BasicBlock, self).__init__()      # super()调用父类

        """
        conv2d -> batchnormalization -> relu activation
        """
        #unit1
        self.conv1 = layers.Conv2D(filters=filter_num, kernel_size=(3, 3), strides=strides, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.Activation('relu')

        #unit2
        self.conv2 = layers.Conv2D(filter_num, (3, 3), strides=1, padding='same')
        self.bn2 = layers.BatchNormalization()

        if strides != 1:
            self.downsample = Sequential()
            self.downsample.add(layers.Conv2D(filter_num, (1, 1), strides=strides))
        else:
            self.downsample = lambda x : x

    def call(self, inputs, training=None):
        x = inputs
        # 前向传播
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        # 下采样
        down = self.downsample(x)
        # f(x)+x  对张量求和
        out_put = layers.add([out, down])
        out_put = tf.nn.relu(out_put)
        return out_put

In [6]:
class ResNet(keras.Model):
    def __init__(self, layers_dims, num_classes=10):
        super(ResNet, self).__init__()

        #预处理层
        self.stem = Sequential([layers.Conv2D(64, (3, 3), strides=(1, 1)),
                                layers.BatchNormalization(),
                                layers.Activation('relu'),
                                layers.MaxPool2D((2, 2), strides=(1, 1), padding='same')])
        #接上4个ResBlock层
        self.resblock1 = self.ResBlock(64, blocks=layers_dims[0])
        self.resblock2 = self.ResBlock(128, blocks=layers_dims[1], strides=2)
        self.resblock3 = self.ResBlock(256, blocks=layers_dims[2], strides=2)
        self.resblock4 = self.ResBlock(512, blocks=layers_dims[3], strides=2)

        #分类层
        self.avgpool = layers.GlobalAveragePooling2D()
        self.fc = layers.Dense(num_classes)

    def call(self, inputs, training=None):
        out = self.stem(inputs)
        out = self.resblock1(out)
        out = self.resblock2(out)
        out = self.resblock3(out)
        out = self.resblock4(out)
        out = self.avgpool(out)
        out = self.fc(out)

        return out

    def ResBlock(self, filter_nums, blocks, strides=1):
        resblock = Sequential()
        resblock.add(BasicBlock(filter_nums, strides))
        #  _ 在for循环中只是一个循环标志 类似于i, j
        for _ in range(blocks):
            resblock.add(BasicBlock(filter_nums, strides=1))
        return resblock

In [8]:
# ResNet-18 18层卷积层   1+4*2*2+1   一个ResBlock包含两个BasicBlock，一个BasicBlock包含两个卷积层
resnet_18 = ResNet([2, 2, 2, 2])


# 测试网络输出shape
# x = tf.random.normal((1, 32, 32, 3))
# out = resnet_18(x)
# print(out.shape)

# 输出网络结构
resnet_18.build(input_shape=(None, 32, 32, 3))
# 输出参数 Param 计算过程
resnet_18.summary()

Model: "res_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 30, 30, 64)        2048      
                                                                 
 sequential_1 (Sequential)   (None, 30, 30, 64)        223104    
                                                                 
 sequential_2 (Sequential)   (None, 15, 15, 128)       823168    
                                                                 
 sequential_4 (Sequential)   (None, 8, 8, 256)         3284736   
                                                                 
 sequential_6 (Sequential)   (None, 4, 4, 512)         13123072  
                                                                 
 global_average_pooling2d (G  multiple                 0         
 lobalAveragePooling2D)                                          
                                                           

In [9]:
def main():
    for epoch in range(epochs):
        for step, (x, y) in enumerate(train_db):       # 在训练集上训练 train_db
            with tf.GradientTape() as tape:
                logits = resnet_18(x)   # logits是网络输出层的输出
                y_onehot = tf.one_hot(y, depth=10)   # 一维向量，标签
                # tf.losses.categorical_crossentropy   先是正确值 再是预测值 否则loss优化会出错
                loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
                loss = tf.reduce_mean(loss)
            grads = tape.gradient(loss, resnet_18.trainable_variables)
            optimizer.apply_gradients(zip(grads, resnet_18.trainable_variables))


            if step % 10 == 0:
                print(epoch, step, 'loss:', float(loss))

            """
            ##  tf.argmax(input, axis)会根据axis取值的不同返回每行或者每列最大值的索引，
            #  当axis=0时：比较每一列的元素，输出每一列最大元素所在的索引数组
            #  当axis=1时：比较每一行的元素，输出每一行最大元素所在的索引数组
            ## tf.equal(x, y) 用来判断两个矩阵或者向量相等的元素，相等返回true，反之返回false，返回的值得矩阵的维度=x=y的维度
            ## tf.reduce_mean() 用来计算张量(tensor)沿着指定的数轴(tensor的某一维度)上的平均值，主要用于降维或者计算tensor的平均值
            """
            if step % 50 == 0:
                total_correct = 0
                total_num = 0
                for step, (x, y) in enumerate(test_db):         # 测试集 test_db
                    logits = resnet_18(x)   # resnet网络的输出结果
                    prob = tf.nn.softmax(logits, axis=1)    ## 经过softmax以后的 概率
                    pred = tf.cast(tf.argmax(prob, axis=1), dtype=tf.int32)   ## 每一行的最大概率 ，
                    correct = tf.reduce_sum(tf.cast(tf.equal(pred, y), dtype=tf.int32))

                    total_correct += correct
                    total_num += x.shape[0]
                acc = total_correct/total_num    # 计算 准确率
                print(epoch, step, 'acc:', float(acc))
                resnet_18.save_weights('./checkpoint/weights.ckpt')
                print('save weights')


if __name__ == '__main__':
    main()

0 0 loss: 2.321587562561035
0 78 acc: 0.1
save weights
0 10 loss: 2.2805662155151367
0 20 loss: 2.2072219848632812
0 30 loss: 2.1455698013305664
0 40 loss: 2.1353025436401367
0 50 loss: 2.0188426971435547
0 78 acc: 0.2423
save weights
0 60 loss: 1.9447022676467896
0 70 loss: 1.8268839120864868
0 80 loss: 1.8992937803268433
0 90 loss: 1.7223906517028809
0 100 loss: 1.9277873039245605
0 78 acc: 0.3011
save weights
0 110 loss: 1.7918689250946045
0 120 loss: 1.8325366973876953
0 130 loss: 1.9281809329986572
0 140 loss: 1.6651159524917603
0 150 loss: 1.634725570678711
0 78 acc: 0.3739
save weights
0 160 loss: 1.5836471319198608
0 170 loss: 1.7048349380493164
0 180 loss: 1.6841905117034912
0 190 loss: 1.506029486656189
0 200 loss: 1.6742441654205322
0 78 acc: 0.4013
save weights
0 210 loss: 1.6577266454696655
0 220 loss: 1.6538832187652588
0 230 loss: 1.6991896629333496
0 240 loss: 1.6285042762756348
0 250 loss: 1.6547770500183105
0 78 acc: 0.3985
save weights
0 260 loss: 1.4578973054885864


KeyboardInterrupt: ignored